install necessary packages and relative inputs 

In [1]:
import pandas as pd

from momentum_backtester.adapters.sp500_github_adapter import load_tiny_sample, load_sp500_data_wrds
from momentum_backtester.backtester import Backtester
from momentum_backtester.signals import price_momentum
from momentum_backtester.ranking import cross_sectional_rank
from momentum_backtester.aggregation import long_short_top_bottom_sector_neutral
from momentum_backtester.costs import turnover_costs
from momentum_backtester.analysis import cagr, annual_vol, sharpe, max_drawdown

First, we load in needed data

In [3]:
data = load_sp500_data_wrds(start_year=2022, end_year=2024)

Loading SP500 data...
Connecting to WRDS with username: zhenggong123
Loading library list...
Done
Connected to WRDS!
Loading SP500 data for year 2022...
the number of unique permnos for year 2022 is 503
the number of unique gvkeys for year 2022 is 498
Loading SP500 data for year 2023...
the number of unique permnos for year 2023 is 501
the number of unique gvkeys for year 2023 is 498


/Users/zhenggong/equity_longshort_backtester/src/momentum_backtester/adapters/sp500_github_adapter.py:52: FutureWarning: The default fill_method='pad' in Series.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  price_df_long['ret_oto'] = price_df_long.groupby('permno')['adjopen'].transform(lambda x: x.pct_change())


In [4]:
# let's have a look at the data
# universe
sp500_universes = data["sp500_universes"][0]
sp500_universes

SP500Universe(year=2022, gvkeys=<StringArray>
['012142', '012141', '012138', '001300', '001722', '012635', '012850',
 '003144', '013421', '003413',
 ...
 '179534', '179437', '010903', '180711', '180652', '004016', '183377',
 '183736', '184500', '184996']
Length: 498, dtype: string, permnos=<IntegerArray>
[10104, 10107, 10138, 10145, 10516, 10696, 10909, 11308, 11403, 11404,
 ...
 92614, 92655, 92778, 93002, 93089, 93096, 93132, 93246, 93429, 93436]
Length: 503, dtype: Int64)

In [5]:
# let's have a look at the data
# return df 
# we have two types of return_df:
# 1. retoto_df_wide: open-to-open return
# 2. retctc_df_wide: close-to-close return
retoto_df_wide = data["retoto_df_wide"]
retctc_df_wide = data["retctc_df_wide"]

# price df
# we have two types of price_df:
# 1. adjclose_df_wide: adjusted close price
# 2. adjopen_df_wide: adjusted open price
adjclose_df_wide = data["adjclose_df_wide"]
adjopen_df_wide = data["adjopen_df_wide"]

# sector df
sector_df_wide = data["sector_df_wide"]

# let's have a look at the data
# retoto_df_wide.tail()
# retctc_df_wide.tail()
adjclose_df_wide.tail()
# adjopen_df_wide.tail()
# sector_df_wide.tail()

permno,10104,10107,10138,10145,10516,10696,10909,11308,11403,11404,...,92614,92655,92778,93002,93089,93096,93132,93246,93429,93436
date,,,,,,,,,,,,,,,,,,,,,
2023-12-22,106.2,374.57999,107.89,205.64,71.46,133.61,<NA>,58.32,275.82001,89.68,...,131.56,520.31,<NA>,112.197998,236.2,132.21001,59.27,128.81,175.53,252.53999
2023-12-26,106.19,374.66,108.69,208.03999,72.39,132.59,<NA>,58.56,274.95999,90.27,...,132.27,520.03003,<NA>,113.189001,237.03,133.06,59.8,129.89,173.69,256.60999
2023-12-27,105.94,374.07001,108.69,209.02,72.17,133.00999,<NA>,58.71,274.64001,89.8,...,132.23,522.78998,<NA>,112.617004,237.22,134.44,59.34,130.28999,175.61,261.44
2023-12-28,106.25,375.28,109.01,209.17,72.27,133.38,<NA>,58.75,273.23999,90.65,...,132.98,524.90002,<NA>,112.241003,237.61,135.55,59.4,130.92999,177.84,253.17999
2023-12-29,105.43,376.04001,107.69,209.71001,72.22,132.84,<NA>,58.93,272.37,90.97,...,131.99001,526.46997,<NA>,111.625,238.86,135.95,58.53,129.24001,178.56,248.48


We now run the backtester

In [6]:
bt = Backtester(
    retoto_df_wide=retoto_df_wide,
    retctc_df_wide=retctc_df_wide,
    adjclose_df_wide=adjclose_df_wide,
    adjopen_df_wide=adjopen_df_wide,
    sector_df_wide=sector_df_wide,
    signal=lambda px: price_momentum(
        px, 
        lookback_months=11, 
        skip=1),
    ranker=cross_sectional_rank,
    aggregator=lambda ranks, sectors: long_short_top_bottom_sector_neutral(
        ranks, 
        sectors, 
        top_pctg=20, 
        bottom_pctg=20),
    costs=lambda w: turnover_costs(w, 10.0),
    rebal_freq="M",
)
results = bt.run()

Analysis

In [7]:
cagr(results["net_returns"], 252)
annual_vol(results["net_returns"], 252)
sharpe(results["net_returns"], 0.04, 252)
max_drawdown(results["net_returns"])

the CAGR is:  0.0135019075001106


NameError: name 'volatility' is not defined